In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Define currencies and their CoinGecko IDs
currencies = {
    "Ether": "ethereum",
    "Akamaru": "akamaru",
    "Carmine Token": "carmine-token",
    "Dai Stablecoin": "dai",
    "Dai Stablecoin (old)": "dai-old",
    "Ekubo Protocol": "ekubo-protocol",
    "LORDS": "lords",
    "LUSD": "liquity-usd",
    "Nostra": "nostra",
    "OWL": "owl",
    "Pain au Lait": "pain-au-lait",
    "Paper": "paper",
    "Rocket Pool ETH": "rocket-pool-eth",
    "Stark Pepe": "stark-pepe",
    "Starknet": "starknet",
    "Standard Weighted Adalian Yield": "standard-weighted-adalian-yield",
    "USDCoin": "usd-coin",
    "Tether USD": "tether",
    "Wrapped BTC": "wrapped-bitcoin",
    "Wrapped stETH": "wrapped-steth"
}

def fetch_currency_data(coin_id, days, currency="usd"):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    response = requests.get(url, params={"vs_currency": currency, "days": days})
    if response.status_code == 200:
        data = response.json()["prices"]
        df = pd.DataFrame(data, columns=["timestamp", "price"])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
        return df
    else:
        raise Exception(f"Failed to fetch data for {coin_id}: {response.status_code}")

# Fetch 100 days of data for all currencies
currency_data = {}
for name, coin_id in currencies.items():
    currency_data[name] = fetch_currency_data(coin_id, days=100)


Exception: Failed to fetch data for owl: 429

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import joblib

def train_and_save_models(currency_data):
    for name, df in currency_data.items():
        print(f"Training model for {name}...")
        train_data = df["price"].values
        model = ARIMA(train_data, order=(5, 1, 0))  # Adjust order as needed
        model_fit = model.fit()
        joblib.dump(model_fit, f"{name}_model.pkl")
        print(f"Model for {name} saved.")

train_and_save_models(currency_data)


In [ ]:
def incremental_train_and_predict(currency_data, days_to_predict=10):
    predictions = {}
    for name, df in currency_data.items():
        print(f"Loading model for {name}...")
        model = joblib.load(f"{name}_model.pkl")
        
        # Use the last 30 days for incremental training
        incremental_data = df["price"].values[-30:]
        model = model.append(incremental_data, refit=True)
        
        # Predict future prices
        forecast = model.forecast(steps=days_to_predict)
        predictions[name] = forecast
        print(f"Prediction for {name}: {forecast}")
    
    return predictions

# Perform incremental training and predictions
predictions = incremental_train_and_predict(currency_data)


In [ ]:
import matplotlib.pyplot as plt

def display_predictions(predictions):
    for name, forecast in predictions.items():
        plt.figure(figsize=(10, 5))
        plt.plot(forecast, label="Predicted Prices", marker="o")
        plt.title(f"Future Price Prediction for {name}")
        plt.xlabel("Days")
        plt.ylabel("Price")
        plt.legend()
        plt.show()

display_predictions(predictions)
